In [ ]:
import streamlit as st
import google.generativeai as genai
import pandas as pd
from PIL import Image
import json

# --- Configuration ---
# IMPORTANT: Configure your Google API key.
# You can get a key from https://aistudio.google.com/app/apikey
# It's recommended to set this as an environment variable for security.
# For Streamlit, you can set it in your secrets.toml file.
# For this example, we'll allow the user to enter it in the sidebar.

st.set_page_config(page_title="Receipt Analyzer", layout="wide")

st.title("🧾 Supermarket Receipt Analyzer")
st.write("Upload an image of a UK supermarket receipt to extract line items, prices, and categories into a table.")

# --- API Key Input ---
st.sidebar.header("Configuration")
api_key = st.sidebar.text_input("Enter your Google API Key", type="password")

if api_key:
    try:
        genai.configure(api_key=api_key)
    except Exception as e:
        st.error(f"Error configuring Google API: {e}")
        st.stop()
else:
    st.info("Please enter your Google API Key in the sidebar to begin.")
    st.stop()


# --- Function to call Gemini API ---
def analyze_receipt(image_file):
    """
    Analyzes a receipt image using the Gemini Pro Vision model.
    """
    if not image_file:
        return None

    try:
        model = genai.GenerativeModel('gemini-pro-vision')
        
        # Open the image file
        img = Image.open(image_file)

        # The prompt is crucial. It instructs the model to act as an expert
        # receipt parser and to return structured JSON.
        prompt = """
        You are an expert receipt analyzer specializing in UK supermarket receipts.
        Analyze the provided receipt image and extract the following information for each line item:
        1.  'item': The full description of the product purchased.
        2.  'price': The price of the item as a float.
        3.  'category': A relevant category for the item (e.g., 'Groceries', 'Fruit & Veg', 'Snacks', 'Drinks', 'Household', 'Toiletries', 'Bakery').

        Please return the output as a single valid JSON object containing a single key "items" which is a list of all extracted items.
        Do not include non-product lines like 'TOTAL', 'Subtotal', 'Card Payment', 'VAT', or any multi-buy savings lines.
        Focus only on the actual products purchased.

        Example format:
        {
          "items": [
            { "item": "TESCO BANANAS 5 PACK", "price": 1.20, "category": "Fruit & Veg" },
            { "item": "Hovis Wholemeal Bread", "price": 1.50, "category": "Bakery" }
          ]
        }
        """

        # Make the API call
        response = model.generate_content([prompt, img])
        
        # Clean up the response to extract only the JSON part
        # LLMs can sometimes add markdown formatting (```json ... ```)
        cleaned_response = response.text.strip().replace('```json', '').replace('```', '')
        
        return json.loads(cleaned_response)

    except Exception as e:
        st.error(f"An error occurred during analysis: {e}")
        st.error("Please ensure your API key is correct and has access to the Gemini API.")
        # Print the raw response for debugging if something goes wrong
        st.text_area("Raw API Response for Debugging", value=response.text if 'response' in locals() else "No response received.", height=200)
        return None

# --- Main App Logic ---
uploaded_file = st.file_uploader("Choose a receipt image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    # Display the uploaded image
    st.image(uploaded_file, caption='Uploaded Receipt', width=350)

    # Analyze the receipt when the button is clicked
    if st.button('Analyze Receipt'):
        with st.spinner('Analyzing... This may take a moment.'):
            result_data = analyze_receipt(uploaded_file)

            if result_data and "items" in result_data:
                st.success("Analysis complete!")
                
                # Convert the list of items to a pandas DataFrame
                df = pd.DataFrame(result_data["items"])
                
                # --- Display DataFrame and Summary ---
                st.subheader("Extracted Items")
                st.dataframe(df)

                st.subheader("Expense Summary")
                total_spent = df['price'].sum()
                
                # Use columns for a cleaner layout
                col1, col2 = st.columns(2)
                col1.metric("Total Items", len(df))
                col2.metric("Total Spent", f"£{total_spent:.2f}")

                # --- Category Breakdown ---
                st.subheader("Spending by Category")
                category_summary = df.groupby('category')['price'].sum().sort_values(ascending=False)
                st.bar_chart(category_summary)

            else:
                st.error("Could not extract items from the receipt. The format might be unsupported or the image quality is low.")
